# For creating Rag application

1. Data
2. Embedding Model
3. Database
4. prompt
5. LLM

Connect all the things

1. Huggingface hub using hf token (inferencing)
2. Load the model in local memory from the hugging face(finetune)

For 2nd approach -  we need lots of memory is required(7b, 10b, 50b, 100b)

so we can use quantized version of the model using bitsandbytes.

3. GGML(GPT-Generated model language), GGUF(GPT-Generated Unified Format)
4. OLLama, Llama-cpp, LM-Studio

In [1]:
%pip intall llama-index-llms-huggingface
%pip install llama-index

ERROR: unknown command "intall" - maybe you meant "install"


In [2]:
!pip install weaviate-client

In [3]:
WEAVIATE_CLUSTER = "https://opensourcellm-p5slkzh4.weaviate.network"
WEAVIATE_API_KEY = "fUg6L4wBbGjYs0ckXsnPtx2TeZVotvFrnhgV"

In [4]:
import weaviate

client = weaviate.Client(url=WEAVIATE_CLUSTER, auth_client_secret = weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [6]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [7]:
loader = SimpleDirectoryReader('/content/Data')

In [8]:
documents = loader.load_data()

In [9]:
documents

[Document(id_='b9498616-44e7-4836-838e-56d209cccead', embedding=None, metadata={'page_label': '1', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': '/content/Data/MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482134, 'creation_date': '2024-05-05', 'last_modified_date': '2024-05-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/355917108\nNeu ral Machine T ranslation with Attention\nTechnic al R eport  · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATIONS\n0READS\n1,579\n2 author s:\nMohammad W asil Saleem\nUniv ersität P otsdam\n3 PUBLICA TIONS \xa0\xa0\xa00 CITATIONS \xa0\xa

In [10]:
%pip install llama-index-embeddings-huggingface
%pip install llama-embeddings-instructor

ERROR: Could not find a version that satisfies the requirement llama-embeddings-instructor (from versions: none)
ERROR: No matching distribution found for llama-embeddings-instructor


In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
from llama_index.core import PromptTemplate


In [13]:
pip install llama-index-llms-huggingface

In [14]:
from llama_index.llms.huggingface import HuggingFaceLLM

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [15]:
system_prompt=""""<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided

"""

In [16]:
# this will wrap the default prompts that are internal to Llama-index

query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [17]:
pip install huggingface_hub

In [18]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [37]:
!pip install accelerate

In [19]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}

)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [21]:
from llama_index.core import ServiceContext

Service_Context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-21-ea21bb51a92a>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context = ServiceContext.from_defaults(


In [22]:
from llama_index.core import VectorStoreIndex

In [23]:
index = VectorStoreIndex.from_documents(documents, service_context=Service_Context)

In [24]:
query_engine = index.as_query_engine(k=1)

In [25]:
result = query_engine.query("Which metrics we can use for evaluating the machines translation task?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
result.response

'We can use BLEU (Bilingual Evaluation Understudy) score as a metric for evaluating machine translation task. BLEU score measures the similarity between the machine translation and a set of human reference translations. The higher the BLEU score, the better the machine translation. Other metrics that can be used include perplexity, which measures the probability of the machine translation given the source text, and ROUGE (Recall-Oriented Understudy for Gisting Evaluation), which measures the overlap between the machine translation and a set of human reference translations.'

In [27]:
result

Response(response='We can use BLEU (Bilingual Evaluation Understudy) score as a metric for evaluating machine translation task. BLEU score measures the similarity between the machine translation and a set of human reference translations. The higher the BLEU score, the better the machine translation. Other metrics that can be used include perplexity, which measures the probability of the machine translation given the source text, and ROUGE (Recall-Oriented Understudy for Gisting Evaluation), which measures the overlap between the machine translation and a set of human reference translations.', source_nodes=[NodeWithScore(node=TextNode(id_='885da3e0-9573-45c4-b68c-749071bd4cfe', embedding=None, metadata={'page_label': '7', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': '/content/Data/MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482134, 'creation_date': '2024-05-05', 'last_modified_date': '2024-05-05'}, excluded_embed_metadata_keys=[

In [28]:
print(result)

We can use BLEU (Bilingual Evaluation Understudy) score as a metric for evaluating machine translation task. BLEU score measures the similarity between the machine translation and a set of human reference translations. The higher the BLEU score, the better the machine translation. Other metrics that can be used include perplexity, which measures the probability of the machine translation given the source text, and ROUGE (Recall-Oriented Understudy for Gisting Evaluation), which measures the overlap between the machine translation and a set of human reference translations.


In [29]:
result2 = query_engine.query("What is attention machenism ?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [30]:
result2.response

'Attention mechanism is a technique used in machine translation to improve the accuracy of the translation by retaining and utilizing all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which leads to better translation of the source sentence. The attention mechanism works by creating an alignment between each time step of the decoder output and all of the encoder hidden state. The alignment score is calculated by multiplying the decoder hidden state at time step t with all of the encoder hidden state, which gives us the alignment score of each of the encoder output with respect to the decoder input and hidden state at that time step. The alignment score is then passed through the softmax layer to calculate the probability distribution, and each of the attention weights is multiplied with each of the encoder hidden state to get the context vector. The context vector is then

In [31]:
print(result2)

Attention mechanism is a technique used in machine translation to improve the accuracy of the translation by retaining and utilizing all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which leads to better translation of the source sentence. The attention mechanism works by creating an alignment between each time step of the decoder output and all of the encoder hidden state. The alignment score is calculated by multiplying the decoder hidden state at time step t with all of the encoder hidden state, which gives us the alignment score of each of the encoder output with respect to the decoder input and hidden state at that time step. The alignment score is then passed through the softmax layer to calculate the probability distribution, and each of the attention weights is multiplied with each of the encoder hidden state to get the context vector. The context vector is then 

In [32]:
from IPython.display import Markdown, display

In [34]:
display(Markdown(f"<b>{result}</b>"))

<b>We can use BLEU (Bilingual Evaluation Understudy) score as a metric for evaluating machine translation task. BLEU score measures the similarity between the machine translation and a set of human reference translations. The higher the BLEU score, the better the machine translation. Other metrics that can be used include perplexity, which measures the probability of the machine translation given the source text, and ROUGE (Recall-Oriented Understudy for Gisting Evaluation), which measures the overlap between the machine translation and a set of human reference translations.</b>

# Below code for Weaviate Cloud storage and above is for In-memory storage

In [35]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [36]:
!pip install llama-index-vector-stores-weaviate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.5.7
    Uninstalling weaviate-client-4.5.7:
      Successfully uninstalled weaviate-client-4.5.7


In [37]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [38]:
from llama_index.core import StorageContext

In [39]:
vector_store=WeaviateVectorStore(weaviate_client=client,index_name="Llamaindex")

In [40]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [41]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,embed_model=embed_model
)


In [42]:
query_engine = index.as_query_engine(llm=llm)

In [43]:
result3=query_engine.query("what is attention?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [44]:
result3.response

'Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence when producing the output. The attention mechanism just addresses the issue of the encoder not being able to memorize the words coming at the beginning of the sentences, which leads to poor translation of the source sentence. During the decoding phase, the model creates an alignment between each time step of the decoder output and all of the encoder hidden state. We need to learn this alignment. Each output of the decoder can selectively pick out specific elements from the sequence to produce the output. This allows the model to focus and pay more attention to the relevant part of the input sequence.'